# Agents Unplugged - for Real, no magic, no fuss
## LangChain, LangFlow and LangGraph

In [3]:
import json, os
file_name = 'config.json'  #paste path here
with open(file_name, 'r') as file:
    config = json.load(file)
    API_KEY = config.get("API_KEY") # Loading the API Key
    OPENAI_API_BASE = config.get("OPENAI_API_BASE") # Loading the API Base Url

In [4]:
# Importing the GPT-4 LLM and setting it up
os.environ['OPENAI_API_KEY']= API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE ;

In [5]:
# Backend config — set ONE line
BACKEND = "OPENAI" # "OLLAMA"  # or "OPENAI"
MODEL_OPENAI = "gpt-4o-mini"
MODEL_OLLAMA = "llama3.1:8b"  # ensure it's pulled if using Ollama

import os, json, re, math, requests, ast, operator
from typing import List, Dict, Any

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

try:
    if BACKEND == "OPENAI":
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=MODEL_OPENAI, temperature=0.2)
    else:
        try:
            from langchain_community.chat_models import ChatOllama
        except Exception:
            from langchain_community.llms import Ollama as ChatOllama
        llm = ChatOllama(model=MODEL_OLLAMA)
except Exception as e:
    raise RuntimeError(f"Could not initialize LLM for BACKEND={BACKEND}: {e}")

llm.invoke("How are you doing today?")  # test if LLM is working

/home/asurace/github/Agents unplugged_ real_no_magic_no_fuss/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content="I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 13, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CK4ZYsG4GvyGSOWLfDJecSvDF7vSn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--af7deb07-fae0-41d8-80f0-bd4fcaabdcf4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 27, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# Tiny policy corpus + tools
POLICIES = [
    {"id": "TravelPlus-2024", "text": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."},
    {"id": "DeviceCare-Pro", "text": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."},
    {"id": "FleetAssist", "text": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."},
]

def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

def search_policies(query: str) -> str:
    hits = simple_search(query, top_k=3)
    return json.dumps({"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]})

# Safe calculator tool using ast
OPS = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv,
       ast.Pow: operator.pow, ast.USub: operator.neg, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv}

def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.BinOp): return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp): return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")

def calculator(expression: str) -> str:
    try:
        node = ast.parse(expression, mode='eval').body
        return str(_eval(node))
    except Exception as e:
        return f"Error: {e}"

tools = [
    Tool(name="search_policies", func=search_policies, description="Search a tiny policy corpus. Returns JSON."),
    Tool(name="calculator", func=calculator, description="Evaluate arithmetic like '2*(1500-200)'. Returns a number as text.")
]


# ReAct agent (ZERO_SHOT_REACT_DESCRIPTION) 

determines the goal directly from the questions without additional context. Here's how it works:

The agent understands the questions using:

Built-in prompt template: The ZERO_SHOT_REACT_DESCRIPTION agent uses a default prompt that instructs the LLM to:

Analyze the question
Reason about what information is needed
Use available tools to find answers
Formulate a response
Tool knowledge: The agent has access to tools (defined in the tools parameter) - search and calculator.

ReAct framework: The agent follows a "Reasoning + Acting" pattern where it:

Thinks about what the question is asking
Determines which tool to use
Uses the tool to get information
Continues reasoning until it reaches an answer
The agent doesn't need explicit contextual guidance because the combination of the question itself, the available tools, and the LLM's capabilities allow it to formulate a plan to answer each insurance-related query.

In [8]:
# Build the agent (ReAct)
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True, handle_parsing_errors=True)

demo_questions = [
    "My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?",
    "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
    "Does FleetAssist cover personal belongings inside a rental car?"
]
print("Demo questions:", *demo_questions, sep="\n- ")


Demo questions:
- My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?
- If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?
- Does FleetAssist cover personal belongings inside a rental car?


/tmp/ipykernel_113865/1077567303.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,


In [9]:
agent.invoke(demo_questions[0])



> Entering new AgentExecutor chain...
I need to find out if the theft of a work smartphone on a business trip is covered under any relevant policies, and if so, what the conditions are. I'll search the policy corpus for information related to coverage for stolen work smartphones during business trips.  
Action: search_policies  
Action Input: "stolen work smartphone business trip coverage"  
Observation: {"results": [{"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}, {"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}]}
Tho

{'input': 'My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?',
 'output': 'Yes, your work smartphone is covered if it was stolen during a business trip. Under the **TravelPlus-2024** policy, personal electronics are covered for theft with a CHF 200 deductible. Under the **DeviceCare-Pro** policy, coverage is available if a police report is filed within 48 hours, with a maximum payout of CHF 1,500 per device and a limit of 2 incidents per policy year.'}

In [10]:
agent.invoke(demo_questions[1])



> Entering new AgentExecutor chain...
To determine the maximum total payout after any deductibles for a laptop worth CHF 1,800 that is accidentally damaged twice in a year under DeviceCare-Pro, I need to find the relevant policy details regarding coverage limits and deductibles for accidental damage. 

Action: search_policies
Action Input: "DeviceCare-Pro accidental damage coverage limits and deductibles"

Observation: {"results": [{"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}, {"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not c

/tmp/ipykernel_113865/1964599603.py:28: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num): return node.n
/tmp/ipykernel_113865/1964599603.py:28: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  if isinstance(node, ast.Num): return node.n


The maximum total payout for the two incidents of accidental damage is CHF 3,000. However, since the policy covers only up to CHF 1,500 per incident, the total payout for both incidents is CHF 3,000. 

Final Answer: CHF 3,000

> Finished chain.


{'input': "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
 'output': 'CHF 3,000'}

In [ ]:
agent.invoke(demo_questions[2])



> Entering new AgentExecutor chain...
I need to check if there are any policies related to FleetAssist that mention coverage for personal belongings inside a rental car. This will help me determine if such coverage exists. 
Action: search_policies
Action Input: "FleetAssist personal belongings rental car coverage"
Observation: {"results": [{"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}, {"id": "FleetAssist", "snippet": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."}, {"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is fil

{'input': 'Does FleetAssist cover personal belongings inside a rental car?',
 'output': 'No, FleetAssist does not cover personal belongings inside a rental car unless explicitly endorsed.'}

# Let's evaluate the questions:

In [11]:
# Micro-evaluator
import pandas as pd
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    def display_dataframe_to_user(name, df):
        print(f"\n[name={name}]\n{df}")

def contains_all(text: str, phrases: list[str]) -> bool:
    low = text.lower()
    return all(p.lower() in low for p in phrases)

tests = [
    {"q": demo_questions[0], "must": ["covered", "theft", "police", "48"]},
    {"q": demo_questions[2], "must": ["not", "unless", "endorsed"]},
]
rows = []
for t in tests:
    out = agent.invoke(t["q"])
    ans = out["output"] if isinstance(out, dict) and "output" in out else str(out)
    rows.append({"question": t["q"], "passed": contains_all(ans, t["must"]), "answer": ans})

df = pd.DataFrame(rows)
display_dataframe_to_user("LangChain agent micro‑eval", df)
df




> Entering new AgentExecutor chain...
I need to find out if a stolen work smartphone is covered under a policy and what the specific conditions are for coverage. I'll search the policy corpus for relevant information regarding coverage for stolen work devices.  
Action: search_policies  
Action Input: "stolen work smartphone coverage"  
Observation: {"results": [{"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}, {"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}]}
Thought:I found two relevant policies regarding coverage fo

,question,passed,answer
0,My work smartphone was stolen on a business tr...,True,"Yes, your stolen work smartphone is covered un..."
1,Does FleetAssist cover personal belongings ins...,True,"No, FleetAssist does not cover personal belong..."


### Call your LangFlow flow (optional)

In [16]:
import requests, json
def run_langflow(flow_id: str, user_input: str, base_url: str = "http://127.0.0.1:7860/api/v1"):
    url = f"{base_url}/run/{flow_id}"
    payload = {"input_value": user_input, "output_type": "chat", "input_type": "chat"}
    r = requests.post(url, json=payload, timeout=60); r.raise_for_status()
    data = r.json()
    try:
        return data["outputs"][0]["outputs"][0]["results"]["message"]["text"]
    except Exception:
        return json.dumps(data, indent=2)

# Example:
print(run_langflow("55e5414b-966a-48dc-8bc4-bfb2e3168c72", "Hello from the notebook! Tell me what I can do with this integration between LangChain and LangFlow"))


HTTPError: 403 Client Error: Forbidden for url: http://127.0.0.1:7860/api/v1/run/55e5414b-966a-48dc-8bc4-bfb2e3168c72